# Устанавливаем библиотеки

In [1]:
!pip install -q openai
!pip install -q tiktoken
!pip install -q python-docx
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 81.9 MB/s eta 0:00:00
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: google-auth-oauthlib 1.0.0
    Uninstalling google-auth-oauthlib-1.0.0:
      Successfully uninstalled google-auth-oauthlib-1.0.0
  Attempting uninstall: google-auth-httplib2
    Found existing installation: google-auth-httplib2 0.1.0
    Uninstalling google-auth-httplib2-0.1.0:
      Successfully uninstalled google-auth-httplib2-0.1.0
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.84.0
    Uninstalling google-api-python-client-2.84.0:
      Successfully uninstalled google-api-python-client-2.84.

In [2]:
import tiktoken
import getpass
import openai
import os
import io
import json
from googleapiclient.discovery import build
from google.colab import auth
from googleapiclient.http import MediaIoBaseDownload
from docx import Document

In [3]:
# Ключ API пользователя и установка его как переменной окружения
openai_key = getpass.getpass("OpenAI API Key:")
os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key = openai_key

OpenAI API Key:··········


# Настройка доступа к Google Диску и получение файлов из папки на Google Диске

In [4]:
auth.authenticate_user()                                                        # Авторизации на доступ к Google Диску
drive_service = build('drive', 'v3')                                            # Создание объект service для доступа к Google Drive API

In [5]:
folder_id = '1hgtn2NTBopGB95ri2oXV3m1tMyK3sFIN'                                 # Идентификатор папки, в которой находятся файлы
results = drive_service.files().list(q=f"'{folder_id}' in parents").execute()   # Список файлов из указанной папки на Google Диске
files = results.get('files', [])
files = [file for file in files if file['name'].endswith('.docx')]              # Удаляем файлы не имеющие расширение 'docx'

In [6]:
files[0]

{'kind': 'drive#file',
 'mimeType': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document',
 'id': '1qT2buEq4yIgAp6Us-2c6iL9imE5n0VOm',
 'name': '1 резюме.docx'}

In [7]:
texts = []

# Пройдем по списку файлов
for file in files:
    file_id = file['id']
    file_name = file['name']

    # Загрузим файл из Google Диска
    request = drive_service.files().get_media(fileId=file_id)
    file_io = io.BytesIO()
    downloader = MediaIoBaseDownload(file_io, request)
    done = False
    while done is False:
        _, done = downloader.next_chunk()

    # Прочитаем текст из файла docx
    document = Document(file_io)
    text = ""
    for paragraph in document.paragraphs:
        text += paragraph.text

    file['text'] = text

In [8]:
files[0]

{'kind': 'drive#file',
 'mimeType': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document',
 'id': '1qT2buEq4yIgAp6Us-2c6iL9imE5n0VOm',
 'name': '1 резюме.docx',
 'text': 'Принципы хорошего резюмеГлобально резюме состоит из трех информационных блоков:* Блок с компаниями, в которых вы работали.* Блок с технической экспертизой: опыт, знания, умения, навыки.* Результаты работы или достижения.Блок с компаниями. Иногда встречаются резюме, в которых ничего нет, кроме упоминания о работе в «крутой» компании, например, в Яндексе.￼Две сухие строки опыта двух лет работы ни о чем говорят. Здесь «крутая» компания играет против кандидата. С таким резюме возникает много вопросов: «А что конкретно он делал два года в Яндекс? За что платить 8 000 евро? За скромный стек? А на каком уровне он им владеет? Чем он будет полезен?» Возможно, это гениальный специалист в своей области, но по резюме это не понятно.Недостаточно написать о работе в крупной корпорации. Это лишь дополнительный бо

# Считаем количество токенов в каждом ключе 'text' списка 'files'

In [9]:
# Создаем экземпляр токенизатора
encoding = tiktoken.get_encoding("cl100k_base")

# Проходим по каждому элементу списка
for file in files:
    token_count = len(encoding.encode(file['text']))                            # Токенизируем текст и считаем количество токенов
    file['token'] = token_count                                                 # Записываем количество токенов в ключ 'token'

    print("name:", file['name'])
    print('token:', file['token'],'\n' )

name: 1 резюме.docx
token: 4942 

name: 16 резюме_.docx
token: 2790 

name: 14 резюме_.docx
token: 3657 

name: 13 резюме_.docx
token: 5620 

name: 10 резюме_.docx
token: 3709 

name: 7 резюме_.docx
token: 7924 

name: 6 резюме_.docx
token: 2551 

name: 17 резюме_.docx
token: 3019 

name: 12 резюме.docx
token: 4793 

name: 11 резюме.docx
token: 2057 

name: 8 резюме.docx
token: 7736 

name: 5 резюме.docx
token: 4195 

name: 4 резюме.docx
token: 4138 

name: 3 резюме.docx
token: 3504 

name: 2 резюме.docx
token: 6041 

name: 15 резюме.docx
token: 4918 

name: 9 резюме.docx
token: 10666 



In [10]:
files[0]

{'kind': 'drive#file',
 'mimeType': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document',
 'id': '1qT2buEq4yIgAp6Us-2c6iL9imE5n0VOm',
 'name': '1 резюме.docx',
 'text': 'Принципы хорошего резюмеГлобально резюме состоит из трех информационных блоков:* Блок с компаниями, в которых вы работали.* Блок с технической экспертизой: опыт, знания, умения, навыки.* Результаты работы или достижения.Блок с компаниями. Иногда встречаются резюме, в которых ничего нет, кроме упоминания о работе в «крутой» компании, например, в Яндексе.￼Две сухие строки опыта двух лет работы ни о чем говорят. Здесь «крутая» компания играет против кандидата. С таким резюме возникает много вопросов: «А что конкретно он делал два года в Яндекс? За что платить 8 000 евро? За скромный стек? А на каком уровне он им владеет? Чем он будет полезен?» Возможно, это гениальный специалист в своей области, но по резюме это не понятно.Недостаточно написать о работе в крупной корпорации. Это лишь дополнительный бо

# Антотации из инструкций по написанию резюме



In [11]:
system_cont = '''
Ты - эксперт и HR-специалист, профессионально составляющий резюме для своих клиентов. Ты отлично структуририруешь текст и выделяешь ключевые моменты.
Возьми на себя роль редактора, которому предоставили текст для получения анотации, и сконцентрируйся только на его содержании, игнорируя вопросы форматирования и дизайна.
В своем анотации выдели из полученного текста основные идеи и требования.
Важно, чтобы анотация содержала список от 5 до 7 пунктов. Пронумеруй все пункты списка.
Выбери только информацию относящуюся к написанию резюме, о спроводительном письме не пиши.
Каждый пункт списка анотации обязательно должен состоять из 3 или 4 предложений. Первое предложение содержит тезис пункта, а следующие предложения раскрывают этот тезис.
Оставь только самые важные пункты по содержанию резюме.
'''

Так, как у большей части инструкций текст больше 4к токенов, то выбирам модель  'gpt-3.5-turbo-16k'

In [13]:
# Проходим по каждому элементу списка
for file in files:
    messages = [{"role": "system", "content": system_cont},
                {"role": "user", "content": file['text']}]                     # f"{file['text']}"}

    completion = openai.ChatCompletion.create(
                  model="gpt-3.5-turbo-16k",
                  messages=messages,
                  temperature=0
                  )
    file['annotation'] = completion['choices'][0]['message']['content']

    print("name:", file['name'])
    print('annotation:', file['annotation'],'\n' )

name: 1 резюме.docx
annotation: Аннотация:

1. Резюме должно состоять из трех информационных блоков: компании, в которых работали; технической экспертизы; результатов работы или достижений.
2. В блоке с компаниями необходимо указывать конкретные задачи и достижения, а не просто упоминать о работе в известной компании.
3. В блоке с технической экспертизой необходимо указывать конкретные технологии и опыт работы с ними, чтобы соответствовать требованиям работодателя.
4. В блоке с достижениями необходимо указывать конкретные результаты и успехи, чтобы продемонстрировать свою ценность как специалиста.
5. Резюме должно быть структурированным и легко читаемым, с разделением на абзацы и смысловые блоки.
6. В резюме также можно указать сопроводительное письмо, в котором рассказать о своей мотивации и почему вы подходите для компании.
7. Важно указывать конкретные обязанности и достижения на проектах, чтобы показать свои навыки и опыт работы.
 

name: 16 резюме_.docx
annotation: 1. "Карандашный

In [18]:
files[0]

{'kind': 'drive#file',
 'mimeType': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document',
 'id': '1qT2buEq4yIgAp6Us-2c6iL9imE5n0VOm',
 'name': '1 резюме.docx',
 'text': 'Принципы хорошего резюмеГлобально резюме состоит из трех информационных блоков:* Блок с компаниями, в которых вы работали.* Блок с технической экспертизой: опыт, знания, умения, навыки.* Результаты работы или достижения.Блок с компаниями. Иногда встречаются резюме, в которых ничего нет, кроме упоминания о работе в «крутой» компании, например, в Яндексе.￼Две сухие строки опыта двух лет работы ни о чем говорят. Здесь «крутая» компания играет против кандидата. С таким резюме возникает много вопросов: «А что конкретно он делал два года в Яндекс? За что платить 8 000 евро? За скромный стек? А на каком уровне он им владеет? Чем он будет полезен?» Возможно, это гениальный специалист в своей области, но по резюме это не понятно.Недостаточно написать о работе в крупной корпорации. Это лишь дополнительный бо

## Считаем количество токенов в анотациях

In [19]:
total_tokens = 0

# Проходим по каждому элементу списка
for file in files:
    token_count = len(encoding.encode(file['annotation']))                      # Токенизируем анотацию и сосчитаем количество токенов
    file['token_anot'] = token_count                                            # Записываем количество токенов в ключ 'token'
    total_tokens += token_count
    print("name:", file['name'])
    print('token:', file['token_anot'],'\n' )

print("total_tokens =", total_tokens)

name: 1 резюме.docx
token: 379 

name: 16 резюме_.docx
token: 446 

name: 14 резюме_.docx
token: 756 

name: 13 резюме_.docx
token: 400 

name: 10 резюме_.docx
token: 710 

name: 7 резюме_.docx
token: 971 

name: 6 резюме_.docx
token: 671 

name: 17 резюме_.docx
token: 424 

name: 12 резюме.docx
token: 605 

name: 11 резюме.docx
token: 635 

name: 8 резюме.docx
token: 384 

name: 5 резюме.docx
token: 937 

name: 4 резюме.docx
token: 343 

name: 3 резюме.docx
token: 340 

name: 2 резюме.docx
token: 753 

name: 15 резюме.docx
token: 913 

name: 9 резюме.docx
token: 264 

total_tokens = 9931


In [20]:
files[0]

{'kind': 'drive#file',
 'mimeType': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document',
 'id': '1qT2buEq4yIgAp6Us-2c6iL9imE5n0VOm',
 'name': '1 резюме.docx',
 'text': 'Принципы хорошего резюмеГлобально резюме состоит из трех информационных блоков:* Блок с компаниями, в которых вы работали.* Блок с технической экспертизой: опыт, знания, умения, навыки.* Результаты работы или достижения.Блок с компаниями. Иногда встречаются резюме, в которых ничего нет, кроме упоминания о работе в «крутой» компании, например, в Яндексе.￼Две сухие строки опыта двух лет работы ни о чем говорят. Здесь «крутая» компания играет против кандидата. С таким резюме возникает много вопросов: «А что конкретно он делал два года в Яндекс? За что платить 8 000 евро? За скромный стек? А на каком уровне он им владеет? Чем он будет полезен?» Возможно, это гениальный специалист в своей области, но по резюме это не понятно.Недостаточно написать о работе в крупной корпорации. Это лишь дополнительный бо

# Попытка №2

In [26]:
files_2 = files.copy()

In [27]:
system_cont = '''
Ты профессиональный копирайтер. У тебя большой опыт работы с Бизнесом в разных сферах.
Ты профессионально составляешь и структурируешь тексты, умеешь выделять важные моменты в каждом тексте.
Твоя цель: Качественно структурировать текст на Русском языке и выделять основные Темы и Подтемы.
Оставь только самые важные пункты по содержанию резюме.
Необходимо уложиться в 800 токенов.
'''

In [28]:
# Проходим по каждому элементу списка
for file in files_2:
    messages = [{"role": "system", "content": system_cont},
                {"role": "user", "content": file['text']}]                     # f"{file['text']}"}

    completion = openai.ChatCompletion.create(
                  model="gpt-3.5-turbo-16k",
                  messages=messages,
                  temperature=0
                  )
    file['annotation'] = completion['choices'][0]['message']['content']

    print("name:", file['name'])
    print('annotation:', file['annotation'],'\n' )

name: 1 резюме.docx
annotation: Принципы хорошего резюме:

1. Блок с компаниями, в которых вы работали:
- Укажите конкретные компании, в которых вы работали, а не только общие названия.
- Опишите свою роль и ответственности в каждой компании.
- Укажите продолжительность работы в каждой компании.

2. Блок с технической экспертизой:
- Укажите свои знания, умения и навыки, связанные с вашей специализацией.
- Укажите конкретные технологии, с которыми вы работали.
- Если есть опыт работы с конкретными фреймворками или инструментами, укажите их.

3. Результаты работы или достижения:
- Укажите конкретные достижения, которые вы получили в своей работе.
- Опишите, что вы сделали лучше, чем раньше, и какие проблемы вы решили.
- Укажите, какие проекты вы выполнили или какие задачи вы успешно реализовали.

4. Обо мне:
- Расскажите о своих основных достоинствах, качествах и рабочих навыках.
- Укажите, с какими фреймворками или технологиями вы работали.

5. Образование и дополнительные курсы:
- Укаж

In [33]:
files_2[0]

{'kind': 'drive#file',
 'mimeType': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document',
 'id': '1qT2buEq4yIgAp6Us-2c6iL9imE5n0VOm',
 'name': '1 резюме.docx',
 'text': 'Принципы хорошего резюмеГлобально резюме состоит из трех информационных блоков:* Блок с компаниями, в которых вы работали.* Блок с технической экспертизой: опыт, знания, умения, навыки.* Результаты работы или достижения.Блок с компаниями. Иногда встречаются резюме, в которых ничего нет, кроме упоминания о работе в «крутой» компании, например, в Яндексе.￼Две сухие строки опыта двух лет работы ни о чем говорят. Здесь «крутая» компания играет против кандидата. С таким резюме возникает много вопросов: «А что конкретно он делал два года в Яндекс? За что платить 8 000 евро? За скромный стек? А на каком уровне он им владеет? Чем он будет полезен?» Возможно, это гениальный специалист в своей области, но по резюме это не понятно.Недостаточно написать о работе в крупной корпорации. Это лишь дополнительный бо

## Считаем количество токенов в анотациях

In [32]:
total_tokens = 0

# Проходим по каждому элементу списка
for file in files_2:
    token_count = len(encoding.encode(file['annotation']))                      # Токенизируем анотацию и сосчитаем количество токенов
    file['token_anot'] = token_count                                            # Записываем количество токенов в ключ 'token'
    total_tokens += token_count
    print("name:", file['name'])
    print('token:', file['token_anot'],'\n' )

print("total_tokens =", total_tokens)

name: 1 резюме.docx
token: 1007 

name: 16 резюме_.docx
token: 343 

name: 14 резюме_.docx
token: 444 

name: 13 резюме_.docx
token: 960 

name: 10 резюме_.docx
token: 901 

name: 7 резюме_.docx
token: 435 

name: 6 резюме_.docx
token: 453 

name: 17 резюме_.docx
token: 1044 

name: 12 резюме.docx
token: 164 

name: 11 резюме.docx
token: 255 

name: 8 резюме.docx
token: 625 

name: 5 резюме.docx
token: 846 

name: 4 резюме.docx
token: 580 

name: 3 резюме.docx
token: 497 

name: 2 резюме.docx
token: 506 

name: 15 резюме.docx
token: 661 

name: 9 резюме.docx
token: 304 

total_tokens = 10025


# Сумаризациия анотаций

## Объединяем все анотации одного примера в единый текст.

In [9]:
annotations = ''

for file in files:
    annotations = annotations + ' ' + file['annotation']

## Сумаризируем сумму анотаций

In [11]:
system_cont = '''
Ты профессиональный копирайтер. Ты профессионально составляешь и структурируешь тексты, умеешь выделять важные моменты в каждом тексте.
Необходимо объединить похожие по смыслу темы или подтемы. При необходимости перефразировать тему или подтему.
Надо откорректировать список, самые значимые темы поднять выше. Количество тем не должно превышать 15.
Каждый тема обязательно должена состоять из 3, 4 или 5 предложений. Первое предложение содержит тезис темы, а следующие предложения раскрывают этот тезис и дополняют его примерами.
'''

In [13]:
messages = [{"role": "system", "content": system_cont},
            {"role": "user", "content": annotations}]
completion = openai.ChatCompletion.create(
              model="gpt-3.5-turbo-16k",
              messages=messages,
              temperature=0
              )
summ_annotation = completion['choices'][0]['message']['content']

In [14]:
summ_annotation

'- Важно подчеркнуть свои навыки в области анализа данных, машинного обучения и статистики.\n- Укажите свой опыт работы с различными инструментами и языками программирования, такими как Python, R, SQL.\n- Опишите свои проекты, в которых вы применяли алгоритмы машинного обучения для решения реальных бизнес-задач.\n- Укажите свои достижения, такие как улучшение производительности моделей, разработка новых алгоритмов или создание инновационных решений.\n- Укажите свое образование в области анализа данных или статистики, а также дополнительные курсы и сертификаты, которые вы получили.\n- Укажите свои навыки в области визуализации данных, статистического анализа и представления результатов.\n- Укажите свои навыки коммуникации и работы в команде, так как это важные аспекты работы Data Scientist.\n- Укажите свои научные публикации или участие в конференциях и семинарах, чтобы подчеркнуть свою экспертизу в области анализа данных.\n- Укажите свои навыки в области предобработки данных, включая о